In [17]:
import warnings
warnings.filterwarnings("ignore")
import string
import nltk
from nltk.stem import WordNetLemmatizer
import pickle
import json

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
#from keras.optimizers import 
import random
from sklearn.model_selection import train_test_split
from keras.models import load_model


In [18]:
words = []
classes = []
docs = []
ignore_words = string.punctuation
data_file = open("Train_Bot1.json").read()
intents = json.loads(data_file)

In [19]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        docs.append((w, intent['tag']))

        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [20]:
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

In [21]:
words

["'s",
 'a',
 'able',
 'activation',
 'ada',
 'adam',
 'aifl',
 'aiml',
 'am',
 'an',
 'angled',
 'ann',
 'anyone',
 'are',
 'area',
 'artificial',
 'backward',
 'bad',
 'bagging',
 'batch',
 'bayes',
 'belong',
 'best',
 'blended',
 'bloody',
 'boosting',
 'bot',
 'buddy',
 'classification',
 'create',
 'cross',
 'cu',
 'cya',
 'day',
 'deep',
 'did',
 'diffult',
 'do',
 'ensemble',
 'epoch',
 'explain',
 'find',
 'first',
 'for',
 'forest',
 'formula',
 'forward',
 'found',
 'from',
 'fuck',
 'function',
 'get',
 'good',
 'goodbye',
 'gradient',
 'great',
 'hate',
 'have',
 'hell',
 'hello',
 'help',
 'helped',
 'heron',
 'hey',
 'hi',
 'hidden',
 'hour',
 'how',
 'hyper',
 'i',
 'imputer',
 'intelligence',
 'is',
 'jerk',
 'joke',
 'knn',
 'later',
 'layer',
 'learner',
 'learning',
 'leaving',
 'listen',
 'logistic',
 'lost',
 'lot',
 'machine',
 'me',
 'ml',
 'my',
 'naive',
 'name',
 'nb',
 'net',
 'network',
 'neural',
 'no',
 'not',
 'of',
 'online',
 'operation',
 'opertions',

In [22]:
pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

In [23]:
training = []
output_empty = [0] * len(classes)

for doc in docs:
    bag = []
    pat_words = doc[0]

    pat_words = [lemmatizer.lemmatize(word.lower()) for word in pat_words]

    for w in words:
        bag.append(1) if w in pat_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

In [24]:
X_train = list(training[:,0])
Y_train = list(training[:,1])
#X_train, x_test, Y_train, y_test = train_test_split(list(training[:,0]), list(training[:,1]), test_size=0.2)

In [25]:
model = Sequential()
model.add(Dense(128, input_shape = (len(X_train[0]),), activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(len(Y_train[0]), activation = 'softmax'))

model.compile(optimizer = 'SGD', loss = 'categorical_crossentropy', metrics = ['accuracy'])
hist = model.fit(np.array(X_train), np.array(Y_train), epochs = 200, batch_size = 5, verbose = 1)

model.save("chatbot.h5", hist)

Epoch 1/200
26/26 [==============================] - 0s 3ms/step - loss: 2.0725 - accuracy: 0.1395
Epoch 2/200
26/26 [==============================] - 0s 3ms/step - loss: 2.0509 - accuracy: 0.2016
Epoch 3/200
26/26 [==============================] - 0s 3ms/step - loss: 2.0182 - accuracy: 0.2016
Epoch 4/200
26/26 [==============================] - 0s 3ms/step - loss: 1.9713 - accuracy: 0.3023
Epoch 5/200
26/26 [==============================] - 0s 3ms/step - loss: 1.9548 - accuracy: 0.2326
Epoch 6/200
26/26 [==============================] - 0s 3ms/step - loss: 1.9494 - accuracy: 0.2481
Epoch 7/200
26/26 [==============================] - 0s 3ms/step - loss: 1.9489 - accuracy: 0.2248
Epoch 8/200
26/26 [==============================] - 0s 3ms/step - loss: 1.9200 - accuracy: 0.2558
Epoch 9/200
26/26 [==============================] - 0s 3ms/step - loss: 1.8989 - accuracy: 0.2713
Epoch 10/200
26/26 [==============================] - 0s 3ms/step - loss: 1.9016 - accuracy: 0.2713
Epoch 11/

In [26]:
saved_model = load_model("chatbot.h5")
saved_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               20736     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 8)                 520       
                                                                 
Total params: 29,512
Trainable params: 29,512
Non-trainable params: 0
_________________________________________________________________


In [27]:
def clean_up_sequence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [28]:
def bow(sentence, words, show_details = True):
    sentence_words = clean_up_sequence(sentence)

    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("Found in bag: %s" % w)
    return(np.array(bag))

In [29]:
def predict_class(sentence, model):
    p = bow(sentence, words, show_details = False)
    res = model.predict(np.array([p]))[0]
    error = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>error]

    results.sort(key = lambda x: x[1], reverse = True)
    return_list = []

    for r in results:
        return_list.append({"intent": classes[r[0]], "probability":str(r[1])})
    
    return return_list

In [30]:
def getResponse(ints, intents_json):
    
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag'] == tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res


In [31]:
def start_chat():
    print("iNeuron: Hello! How can I help you?\n\n")
    while True:
        inp = str(input()).lower()
        if inp.lower() == "end":
            break
        if inp.lower() == '' or inp.lower() == '*':
            print("Please re-type your query")
            print("-"*50)
        else:
            print(f"iNeuron: {chatbot_response(inp)}" + '\n')
            print("-" * 50)
    

In [ ]:
start_chat()